In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
data = pd.read_csv('/content/train (1).csv')   # name must match uploaded file
data = data[['tweet', 'label']]      # keep only relevant columns
data.columns = ['text', 'category']  # rename columns
print(data.head())

                                                text  category
0   @user when a father is dysfunctional and is s...         0
1  @user @user thanks for #lyft credit i can't us...         0
2                                bihday your majesty         0
3  #model   i love u take with u all the time in ...         0
4             factsguide: society now    #motivation         0


In [ ]:
# 3️) Data Preprocessing
# -------------------------------
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text)   # remove symbols, numbers
    text = text.lower()                      # lowercase
    text = text.split()
    text = [word for word in text if word not in stop_words]
    return ' '.join(text)

data['clean_text'] = data['text'].apply(clean_text)

# Encode labels (convert to numbers)
label_encoder = LabelEncoder()
data['category'] = label_encoder.fit_transform(data['category'])

In [ ]:
# 4️) Tokenization & Padding
# -------------------------------
max_words = 5000
max_len = 100
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(data['clean_text'])
X = tokenizer.texts_to_sequences(data['clean_text'])
X = pad_sequences(X, maxlen=max_len)

y = data['category']

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# 5️) Load GloVe Embeddings
# -------------------------------
# You can download glove.6B.100d.txt from https://nlp.stanford.edu/projects/glove/

embedding_index = {}
# For simplicity, let’s skip loading full file (Colab will need path)
# Example for your use:
# with open('glove.6B.100d.txt', encoding='utf8') as f:
#     for line in f:
#         values = line.split()
#         word = values[0]
#         coeffs = np.asarray(values[1:], dtype='float32')
#         embedding_index[word] = coeffs

embedding_dim = 100
embedding_matrix = np.random.randn(max_words, embedding_dim)  # random init for demo

In [ ]:
# 6️) Build LSTM Model
# -------------------------------
model = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len, weights=[embedding_matrix], trainable=True),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │       500,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 500,000 (1.91 MB)

 Trainable params: 500,000 (1.91 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# 7️) Train the Model
# -------------------------------
history = model.fit(X_train, y_train, epochs=1, batch_size=64, validation_data=(X_test, y_test))

400/400 ━━━━━━━━━━━━━━━━━━━━ 141s 353ms/step - accuracy: 0.9324 - loss: 0.2324 - val_accuracy: 0.9456 - val_loss: 0.1619


In [ ]:
# 8️) Evaluate the Model
# -------------------------------
loss, acc = model.evaluate(X_test, y_test)
print(f" Test Accuracy: {acc*100:.2f}%")

200/200 ━━━━━━━━━━━━━━━━━━━━ 10s 50ms/step - accuracy: 0.9502 - loss: 0.1487
✅ Test Accuracy: 94.56%


In [ ]:
# 9️) Predict on New Text
# -------------------------------
def predict_category(text):
    text = clean_text(text)
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_len)
    pred = model.predict(padded)
    label = 'Positive' if pred[0][0] > 0.5 else 'Negative'
    print(f"📰 News: {text}\nPrediction: {label}")

predict_category("Stock market hits record high amid economic optimism")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 775ms/step
📰 News: stock market hits record high amid economic optimism
Prediction: Negative
